# Lesson 4: Deleting Records by ID

# Deleting Records by ID

Welcome to the section on deleting records by ID using Django ORM! In our previous lessons, we learned how to retrieve and update records from the database. Now, we'll focus on how to delete specific records. This is vital for applications where data cleanup is essential, such as task management systems and user databases.

## What You'll Learn
In this lesson, you'll learn how to delete a record from the database using its ID. This allows you to remove specific items, whether they are old tasks, outdated user profiles, or any other data that needs to be cleaned up.

We will continue to use the `Todo` model from our previous lessons:

### Todo Model

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

## Creating the Delete Function

To delete a `Todo` item by its ID, we'll add a new view function in `views.py`. This function will handle `DELETE` requests to remove the `Todo` object:

### Delete View Function

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.shortcuts import get_object_or_404
from .models import Todo

@csrf_exempt
def delete_todo(request, id):
    if request.method == 'DELETE':
        try:
            todo = get_object_or_404(Todo, id=id)  # Alternative: Todo.objects.get(id=id)
            todo.delete()
            return JsonResponse({'message': 'Todo deleted'})
        except:
            return JsonResponse({'message': 'Todo not found'}, status=404)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Explanation

- **Retrieve and Delete:** The function retrieves the `Todo` object by its ID using `get_object_or_404(Todo, id=id)`, and then deletes it from the database using `.delete()`.
- **Response:** If the object is deleted successfully, a JSON response with the message `'Todo deleted'` is returned. If the object is not found, it returns a `404` error with the message `'Todo not found'`.
- **Invalid Method:** If the request is not a `DELETE` request, the function returns an error message `'Invalid request'` with a `400` status.

> Note: We use the `get_object_or_404` shortcut to retrieve the object by its ID. Alternatively, you can use `Todo.objects.get(id=id)` to achieve the same result.

## Why It Matters

Understanding how to delete records is a fundamental part of web development. Here are some practical scenarios where this knowledge is indispensable:

- **Task Management:** In a to-do list application, users should be able to delete tasks that are no longer relevant.
- **User Management:** In user management systems, administrators may need to delete user accounts that are inactive or violate terms of service.
- **Data Cleanup:** Applications often need to periodically remove old or unnecessary data to optimize performance and storage.

By mastering the skill of deleting records, you ensure that your application remains efficient and user-friendly. Users expect the ability to manage their data seamlessly, and your capabilities will help you deliver that experience.

## Ready to Practice?

Excited to put this into practice? Let's move to the practice section where you can apply what you've learned and fine-tune your skills. Happy coding!

## Deleting Todo Tasks by ID

To run this code and observe the deletion process in Django ORM, follow these steps:

### Step 1: Ensure the Environment Setup

Make sure your Django environment is set up correctly. You should have Django installed and running in your project. 

```bash
pip install django
```

Ensure the `Todo` model is migrated to the database:

```bash
python manage.py makemigrations
python manage.py migrate
```

### Step 2: Add Preload and Delete Functions to Views

In your `views.py`, ensure the preload and delete functionality is implemented as follows:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.shortcuts import get_object_or_404
from .models import Todo

def preload(request):
    todos = [('Prepare breakfast', False), ('Go to work', False), ('Attend meeting', False), ('Morning walk', True)]
    for task, completed in todos:
        if not Todo.objects.filter(task=task).exists():
            Todo.objects.create(task=task, completed=completed)

    todos = Todo.objects.all().values()
    return JsonResponse({'message': 'Todos preloaded', 'todos': list(todos)})

@csrf_exempt
def delete_todo(request, id):
    if request.method == 'DELETE':
        try:
            todo = get_object_or_404(Todo, id=id)
            todo.delete()
            return JsonResponse({'message': 'Todo deleted'})
        except:
            return JsonResponse({'message': 'Todo not found'}, status=404)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Step 3: Add URLs to `urls.py`

Update your `urls.py` to include the paths for preloading and deleting tasks:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('preload/', views.preload, name='preload'),
    path('todos/<int:id>/', views.delete_todo, name='delete_todo'),
]
```

### Step 4: Preload Todos and Delete a Task

Once everything is set up, use the provided `requests` code to preload some tasks and delete one by its ID:

```python
import requests

URL = 'http://127.0.0.1:8000'  # Adjust if necessary

# Preload todos
response = requests.get(URL + '/preload/')
print('Preloaded Todos:', response.json())

# Get the first todo's ID
todos = response.json()['todos']
id = todos[0]['id']

# Delete the first todo
response = requests.delete(URL + f'/todos/{id}/')
print('Delete Response:', response.json())
```

### Step 5: Run the Django Server

Start the Django development server:

```bash
python manage.py runserver
```

Once the server is running, execute the Python script to interact with the API.

### Step 6: Review Output

- **Preloaded Todos:** The list of todos is printed after being preloaded into the database.
- **Delete Response:** After sending a `DELETE` request, the response confirming the deletion of the Todo task is printed.

### Expected Output

1. After preloading:

    ```json
    {
        "message": "Todos preloaded",
        "todos": [
            {"id": 1, "task": "Prepare breakfast", "completed": false},
            {"id": 2, "task": "Go to work", "completed": false},
            {"id": 3, "task": "Attend meeting", "completed": false},
            {"id": 4, "task": "Morning walk", "completed": true}
        ]
    }
    ```

2. After deletion:

    ```json
    {
        "message": "Todo deleted"
    }
    ```

## Delete Todo Item by Task Name

To delete a `Todo` item by its **task name** instead of its **ID**, we'll need to modify the views and URLs accordingly. Let's walk through the necessary changes.

### Step 1: Update `delete_todo` View to Delete by Task Name

In `views.py`, modify the `delete_todo` function to accept the task name as a parameter instead of the ID. Use `get_object_or_404` to find the `Todo` by its task name.

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.shortcuts import get_object_or_404
from .models import Todo

@csrf_exempt
def preload(request):
    todos = [('Prepare breakfast', False), ('Go to work', False), ('Attend meeting', False), ('Morning walk', True)]
    for task, completed in todos:
        if not Todo.objects.filter(task=task).exists():
            Todo.objects.create(task=task, completed=completed)

    todos = Todo.objects.all().values()
    return JsonResponse({'message': 'Todos preloaded', 'todos': list(todos)})

# Updated delete_todo to delete by task name
@csrf_exempt
def delete_todo(request, task):
    if request.method == 'DELETE':
        try:
            # Retrieve the Todo object by task name
            todo = get_object_or_404(Todo, task=task)
            todo.delete()
            return JsonResponse({'message': 'Todo deleted'})
        except:
            return JsonResponse({'message': 'Todo not found'}, status=404)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Step 2: Update the `urls.py`

Change the URL pattern to pass the **task name** as a parameter instead of the ID:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('preload/', views.preload, name='preload'),
    # Change to accept task name as a string in the URL
    path('todos/<str:task>/', views.delete_todo, name='delete_todo'),
]
```

### Step 3: Update the `send_request.py`

Update the Python script to delete a `Todo` item by its task name:

```python
import requests

URL = 'http://127.0.0.1:3000'  # Adjust if necessary

# Preload todos
response = requests.get(URL + '/preload/')
print('Preloaded Todos:', response.json())

# Get the first todo's task name
todo = response.json()['todos'][0]
task_name = todo['task']

# Delete the first todo by its task name
response = requests.delete(URL + f'/todos/{task_name}/')
print('Delete Response:', response.json())
```

### Step 4: Run the Django Server and Execute the Requests

Start your Django development server:

```bash
python manage.py runserver
```

Then, run the updated `send_request.py` script to preload and delete a `Todo` item by its task name.

### Step 5: Expected Output

1. After preloading:

    ```json
    {
        "message": "Todos preloaded",
        "todos": [
            {"id": 1, "task": "Prepare breakfast", "completed": false},
            {"id": 2, "task": "Go to work", "completed": false},
            {"id": 3, "task": "Attend meeting", "completed": false},
            {"id": 4, "task": "Morning walk", "completed": true}
        ]
    }
    ```

2. After deletion by task name:

    ```json
    {
        "message": "Todo deleted"
    }
    ```

This approach ensures the deletion process is based on the task name rather than the ID.

## Fix the Task Deletion Bug

To fix the bug where the message incorrectly states that 4 tasks have been deleted instead of 1, you need to adjust how you count and delete the completed tasks in your `delete_completed_todos` view. Here’s how you can correct it:

1. **Counting Completed Tasks**: Use `.count()` instead of `len()` to get the number of completed tasks filtered.

2. **Deleting Completed Tasks**: Ensure you filter only the completed tasks and then delete them.

Here's the corrected version of your `delete_completed_todos` view function:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo

@csrf_exempt
def delete_completed_todos(request):
    if request.method == 'DELETE':
        try:
            completed_todos = Todo.objects.filter(completed=True)
            count = completed_todos.count()  # Correct way to count completed tasks
            completed_todos.delete()  # Delete only the completed tasks
            return JsonResponse({'message': f'{count} completed todos deleted'})
        except Exception as e:
            return JsonResponse({'message': f'Error deleting completed todos: {str(e)}'}, status=500)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Explanation:

- **Filtering Completed Tasks**: `Todo.objects.filter(completed=True)` ensures only tasks marked as completed are fetched.
  
- **Counting Completed Tasks**: `count = completed_todos.count()` accurately counts the number of completed tasks.

- **Deleting Tasks**: `completed_todos.delete()` deletes the tasks that were filtered as completed.

This correction ensures that only the completed tasks are counted and deleted, leading to the correct message being returned indicating how many tasks were actually deleted.

## Deleting Completed Tasks Efficiently

Here's the updated `delete_completed_todos` view function to delete tasks that are marked as completed (`completed=True`) and are not read-only (`read_only=False`).

### Updated `delete_completed_todos` View:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Todo

@csrf_exempt
def delete_completed_todos(request):
    if request.method == 'DELETE':
        try:
            # Filter to get only completed todos that are not read-only
            completed_todos = Todo.objects.filter(completed=True, read_only=False)
            
            # Count the filtered todos
            count = completed_todos.count()
            
            # Delete the filtered todos
            completed_todos.delete()
            
            # Return a response with the number of deleted todos
            return JsonResponse({'message': f'Deleted {count} completed todos'})
        except Exception as e:
            return JsonResponse({'message': f'Error deleting completed todos: {str(e)}'}, status=500)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Key Changes:

1. **Filtering Todos**: The line `completed_todos = Todo.objects.filter(completed=True, read_only=False)` ensures that only tasks marked as completed and not read-only are selected for deletion.

2. **Counting the Filtered Todos**: The number of tasks that meet the criteria is counted using `.count()`.

3. **Deleting the Filtered Todos**: `completed_todos.delete()` deletes the filtered tasks from the database.

4. **Response Message**: The response includes a message like `'Deleted {count} completed todos'`, indicating how many tasks were deleted.

### Example Output:
If there is one completed task that is not read-only, the response would look like:
```json
{
  "message": "Deleted 1 completed todos"
}
```

This approach ensures that tasks marked as read-only are preserved and only completed, non-read-only tasks are deleted.

To add the feature that allows deleting movies by genre in the `delete_movie` view, you need to filter and delete movies based on the genre passed in the request.

Here's the updated `delete_movie` view to include genre-based deletion:

### Updated `delete_movie` View:
```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from .models import Movie

@csrf_exempt
def delete_movie(request, id=None, genre=None):
    if request.method == 'DELETE':
        # Delete by movie ID
        if id:
            try:
                movie = Movie.objects.filter(id=id)
                if not movie.exists():
                    return JsonResponse({'message': 'Movie not found'}, status=404)
                movie.delete()
                return JsonResponse({'message': 'Movie deleted'})
            except Exception as e:
                return JsonResponse({'message': f'Error deleting movie: {str(e)}'}, status=500)

        # Delete by genre
        elif genre:
            try:
                # Filter and retrieve movies by genre
                movies = Movie.objects.filter(genre=genre)
                count = movies.count()
                
                # Handle case where no movies of the given genre are found
                if count == 0:
                    return JsonResponse({'message': 'Movies not found'}, status=404)
                
                # Delete the movies and return the count of deleted movies
                movies.delete()
                return JsonResponse({'message': f'{count} movies of genre {genre} deleted'})
            except Exception as e:
                return JsonResponse({'message': f'Error deleting movies: {str(e)}'}, status=500)

    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Key Points:

1. **Delete by Genre**:
   - The code `movies = Movie.objects.filter(genre=genre)` filters all movies that match the provided genre.
   - `count = movies.count()` counts how many movies will be deleted.
   - If no movies are found, it returns a 404 error message: `'Movies not found'`.
   - Otherwise, it deletes the movies using `movies.delete()` and returns the number of deleted movies in the message.

2. **Error Handling**:
   - The code is wrapped in a `try-except` block to handle unexpected errors.
   - If an error occurs, it returns a 500 status with an appropriate error message.

3. **Fallback for Invalid Requests**:
   - If neither an `id` nor a `genre` is provided in the request, the function returns an invalid request message.

### Example Output:
- If 2 movies of genre 'Sci-Fi' are deleted, the response would be:
```json
{
  "message": "2 movies of genre Sci-Fi deleted"
}
```

- If no movies of genre 'Comedy' are found, the response would be:
```json
{
  "message": "Movies not found",
  "status": 404
}
``` 

This implementation covers both deleting by movie ID and by genre effectively.